You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provided by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [2024-08-04 00:16:44][DEBUG]: == Working Agent: Content Planner
 [2024-08-04 00:16:44][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.  
Final Answer: 

**Content Plan Document on Artificial Intelligence**

---

### 1. Latest Trends in Artificial Intelligence

- **Generative AI**: Recent advancements in generative models, such as OpenAI's ChatGPT and DALL-E, are reshaping content creation, design, and customer interaction.
- **AI Ethics and Governance**: Growing concerns over AI's ethical implications have led to the establishment of guidelines and frameworks by organizations like the OECD and the European Union.
-

- Display the results of your execution as markdown in the notebook.

In [20]:
result.raw

"```markdown\n# The Transformative Power of Artificial Intelligence: Trends, Challenges, and Opportunities\n\n## Introduction\n\nArtificial Intelligence (AI) has become an integral part of the digital landscape, revolutionizing industries and reshaping the way we interact with technology. Its significance is underscored by rapid advancements, particularly in generative models and natural language processing. As we delve into the transformative power of AI, it’s crucial to explore the latest trends, ethical considerations, and the promising opportunities it presents for various sectors.\n\nThe evolution of AI technologies is not just a trend; it is a paradigm shift that influences everything from healthcare to environmental sustainability. In this article, we will examine the latest developments in AI, the key players driving these changes, the challenges we face in implementation, and the future opportunities that lie ahead.\n\n## Latest Trends in AI\n\n### Generative AI: A New Era of 

In [21]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# The Transformative Power of Artificial Intelligence: Trends, Challenges, and Opportunities

## Introduction

Artificial Intelligence (AI) has become an integral part of the digital landscape, revolutionizing industries and reshaping the way we interact with technology. Its significance is underscored by rapid advancements, particularly in generative models and natural language processing. As we delve into the transformative power of AI, it’s crucial to explore the latest trends, ethical considerations, and the promising opportunities it presents for various sectors.

The evolution of AI technologies is not just a trend; it is a paradigm shift that influences everything from healthcare to environmental sustainability. In this article, we will examine the latest developments in AI, the key players driving these changes, the challenges we face in implementation, and the future opportunities that lie ahead.

## Latest Trends in AI

### Generative AI: A New Era of Creation

One of the most compelling advancements in AI is the emergence of generative models like OpenAI’s ChatGPT and DALL-E. These technologies are transforming how content is created across various industries, enabling businesses to automate tasks that were once time-consuming and labor-intensive. Generative AI can produce text, images, and even music, allowing for unprecedented creativity and efficiency. As companies adopt these tools, we are witnessing a shift in customer interaction, where personalized and engaging content is generated on demand.

Moreover, generative AI is not limited to content creation; it is also being applied in design and marketing. By analyzing vast datasets, these models can identify trends and preferences, helping businesses tailor their products and services to meet customer needs effectively. This trend is set to continue, indicating a future where AI will play a central role in creative processes.

### The Rise of AI Ethics and Governance

As AI technologies proliferate, so do the ethical concerns surrounding their use. Issues such as bias in algorithms, data privacy, and accountability have prompted organizations like the OECD and the European Union to establish guidelines and frameworks for responsible AI governance. The establishment of these ethical guidelines is crucial as it ensures that AI innovations do not compromise societal values and human rights.

Incorporating ethical considerations into AI development is not just a regulatory requirement; it is a necessity to foster trust among users and stakeholders. Businesses that prioritize ethical AI practices are likely to gain a competitive edge, as consumers increasingly demand transparency and accountability from the technologies they use.

### Innovations in Healthcare Driven by AI

The healthcare industry is experiencing a significant transformation, driven by advances in AI technologies. From predictive analytics to drug discovery and personalized medicine, AI is enhancing patient care and research capabilities. Companies like IBM Watson Health are at the forefront of these innovations, utilizing AI to analyze vast amounts of medical data, leading to faster diagnoses and improved treatment plans.

AI’s ability to predict patient outcomes and identify potential health risks before they become critical is revolutionizing preventive care. This proactive approach not only enhances patient well-being but also optimizes resource allocation within healthcare systems, making them more efficient and cost-effective.

## Key Players in the AI Industry

### Leading Companies Shaping AI

The landscape of AI is populated by several key players who are pushing the boundaries of what is possible. Companies such as OpenAI, Google, IBM, and Microsoft are investing heavily in AI research and development, contributing to groundbreaking advancements across multiple sectors. Their efforts are not only enhancing existing technologies but also paving the way for new applications that were once considered science fiction.

Emerging startups are also making waves in the AI space. These nimble companies often bring innovative solutions to market faster than their larger counterparts, focusing on niche applications that address specific business challenges. The dynamic interplay between established firms and startups fosters a competitive environment that accelerates technological progress.

### The Importance of Collaboration

Collaboration among industry leaders, startups, and academic institutions is essential for the continued growth of AI technologies. By sharing knowledge and resources, these entities can tackle complex challenges and drive the development of responsible AI solutions. Collaborative initiatives can also help bridge the gap between research and practical application, ensuring that innovations benefit society as a whole.

## Challenges of AI Implementation

### Navigating Ethical Concerns

Despite the promise of AI, implementing these technologies comes with significant challenges. Ethical concerns surrounding bias, accountability, and transparency pose hurdles that organizations must navigate. The potential for AI systems to perpetuate existing biases in data can lead to unfair outcomes, which raises questions about the fairness of automated decision-making processes.

To address these ethical dilemmas, companies must adopt responsible AI practices, including regular audits of algorithms and the inclusion of diverse perspectives in development teams. By prioritizing ethical considerations, businesses can build trust with their stakeholders and create AI systems that align with societal values.

### Technical Hurdles and Workforce Readiness

Technical challenges also hinder AI implementation. Organizations often face difficulties in integrating AI solutions into existing workflows, requiring substantial investments in infrastructure and training. Furthermore, there is a significant skills gap in the workforce, as many professionals lack the necessary knowledge and training to harness AI technologies effectively.

To overcome these barriers, businesses must invest in education and training programs that equip their employees with the skills needed for AI adoption. By fostering a culture of continuous learning, organizations can ensure that their workforce is prepared to navigate the complexities of AI technologies.

## Future Opportunities with AI

### AI in Climate Change Solutions

As the world grapples with climate change, AI presents innovative solutions to monitor environmental changes and optimize energy consumption. By analyzing vast datasets, AI can identify patterns and trends that inform better decision-making and resource management. Companies and governments are increasingly leveraging AI to develop sustainable practices, highlighting the technology's potential to address one of humanity's most pressing challenges.

The application of AI in climate change is not limited to monitoring; it also extends to optimizing supply chains, reducing waste, and improving energy efficiency. These advancements not only benefit the environment but also enhance business operations, demonstrating the dual value of AI in promoting sustainability and profitability.

### AI’s Role in Revolutionizing Business Processes

AI technologies hold the potential to revolutionize business processes across industries. From enhancing customer service through intelligent chatbots to automating routine tasks with robotic process automation, AI can significantly improve operational efficiency. By streamlining processes, businesses can focus on strategic initiatives that drive growth and innovation.

Furthermore, the increasing adoption of AI will create new career opportunities for professionals across various fields. As organizations seek experts in AI development, data analysis, and ethical governance, individuals with the right skill sets will find themselves in high demand. This presents an opportunity for students and educators to explore the vast possibilities that a career in AI can offer.

## Conclusion

The transformative potential of Artificial Intelligence is undeniable. As we continue to witness rapid advancements in AI technologies, it is essential to approach these innovations with a sense of responsibility. By prioritizing ethical considerations and investing in workforce development, we can harness the power of AI to create a more efficient, equitable, and sustainable future.

In an era marked by technological evolution, continuous learning and adaptation will be crucial. The future of AI holds endless possibilities, and it is up to us to shape its trajectory through responsible innovation.

## Call to Action

We encourage you to subscribe for updates on the latest trends in AI and share your thoughts on the challenges and opportunities presented by this transformative technology. Together, let’s engage in a meaningful dialogue about the future of Artificial Intelligence and its impact on our world.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [22]:
topic = "Pandas vs Polars"
result = crew.kickoff(inputs={"topic": topic})

 [2024-08-04 00:20:32][DEBUG]: == Working Agent: Content Planner
 [2024-08-04 00:20:32][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Pandas vs Polars.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer. 

Final Answer:

### Comprehensive Content Plan: Pandas vs Polars

#### 1. Latest Trends, Key Players, and Noteworthy News

- **Trends:**
  - The rise of data processing frameworks that emphasize performance and scalability, such as Polars, in response to the limitations of traditional libraries like Pandas.
  - Increased adoption of Rust-based libraries (Polars is primarily built in Rust) for performance-critical applications, showcasing a trend towards leveraging Rust for data science.
 

In [23]:
Markdown(result.raw)

```markdown
# Pandas vs Polars: Choosing the Right Data Processing Tool for Your Needs

Data manipulation and analysis are at the core of data science, particularly within the Python ecosystem. Two libraries that have significantly influenced this field are Pandas and Polars. While Pandas has long been the go-to tool for data manipulation, Polars is emerging as a formidable alternative, especially in scenarios demanding speed and efficiency. Understanding the nuances between these two libraries is essential for data scientists, engineers, and anyone involved in data-driven decision-making.

In this article, we will delve into the strengths and weaknesses of both Pandas and Polars, providing insights that can help you choose the right tool for your specific needs.

## Section 1: Overview of Pandas

Pandas was developed in 2008 by Wes McKinney as a means to facilitate data analysis in Python. Since its inception, it has become a cornerstone in the data science toolkit, valued for its ease of use and extensive community support. One of the key features of Pandas is its DataFrame, a two-dimensional size-mutable structure that allows users to store and manipulate large datasets efficiently.

Pandas excels in scenarios where quick prototyping and seamless integration into existing Python workflows are essential. Common use cases include exploratory data analysis, data cleaning, and preprocessing tasks. Its vast ecosystem, including libraries like NumPy and Matplotlib, further solidifies its position as a leading data manipulation tool. However, it is important to recognize that performance may falter when dealing with very large datasets, necessitating the exploration of alternative approaches.

## Section 2: Overview of Polars

Polars is a relatively new entrant in the data processing space, developed primarily in Rust, with a strong emphasis on performance and memory efficiency. This architecture allows Polars to perform operations in parallel, making it significantly faster than Pandas for various data manipulation tasks. The library offers a similar API to Pandas, which eases the transition for users familiar with Python data manipulation.

Key features of Polars include its ability to handle larger-than-memory datasets and its efficient execution of complex operations. Use cases where Polars shines include big data applications, real-time data processing, and situations where computational efficiency is critical. As organizations increasingly demand faster processing times for large datasets, Polars' appeal continues to grow.

## Section 3: Performance Comparison

When it comes to performance, benchmarks highlight a stark contrast between Pandas and Polars. For standard tasks such as loading data, filtering, and aggregating, Polars often outperforms Pandas, particularly as dataset sizes increase. In several benchmark studies, Polars demonstrated the capability to handle millions of rows with significantly reduced processing times compared to its counterpart.

Real-world scenarios further illustrate these differences. For instance, a data scientist analyzing extensive user interaction logs may find that using Polars not only accelerates data loading times but also enhances overall workflow efficiency. This performance edge is particularly beneficial in environments where rapid data analysis is crucial for decision-making.

## Section 4: Community and Ecosystem

Both Pandas and Polars boast strong community support and documentation, which are vital for users navigating new tools. Pandas, having been established for over a decade, offers a wealth of resources, tutorials, and community forums that facilitate learning and troubleshooting. The extensive ecosystem surrounding Pandas, including tools like Dask for parallel processing, enhances its capabilities.

Conversely, Polars is quickly gaining traction with a growing community and improving documentation. The integration of Polars with existing data science workflows, including potential collaborations with libraries like Apache Arrow, suggests a promising future. For users looking to explore both libraries, participating in community discussions on platforms like Stack Overflow or Reddit can provide valuable insights and updates.

## Section 5: Making the Choice

Choosing between Pandas and Polars involves considering several factors, such as dataset size, specific features required, and overall ecosystem compatibility. For smaller datasets or projects that require rapid prototyping, Pandas remains a reliable choice due to its user-friendly interface and extensive resources. However, for projects dealing with large datasets or performance-critical applications, Polars may be the more suitable option.

Ultimately, the decision should be informed by the specific demands of your projects. If you're managing a data pipeline that requires quick processing of large volumes of data, Polars could be the ideal fit. Conversely, if you're embarking on exploratory data analysis or smaller-scale projects, the tried-and-true Pandas may serve you better.

## Conclusion

In conclusion, both Pandas and Polars have their unique strengths and weaknesses, catering to different user needs and scenarios. As the landscape of data processing continues to evolve, so too does the necessity for users to stay informed about the tools they choose. Experimenting with both libraries can provide deeper insights into their functionalities, enabling data professionals to make informed decisions tailored to their specific workflows.

We invite you to share your experiences with Pandas and Polars in the comments section below. Additionally, subscribe for more insights into data science tools and techniques, as we continue to explore the dynamic world of data manipulation.
```

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).